### 1. Написати функцію пошуку випадкового простого числа з заданого інтервалу або заданої довжини, використовуючи датчик випадкових чисел та тести перевірки на простоту.

In [6]:
import random

def zgheneruvaty_proste_chyslo(dovzhyna_bita):
    while True:
        chyslo_kandydat = random.getrandbits(dovzhyna_bita) | (1 << dovzhyna_bita - 1) | 1
        if prostye_chyslo(chyslo_kandydat):
            return chyslo_kandydat
        else:
            print(f"Kandydat {chyslo_kandydat} ne proste chyslo.")

def vyznachennya_nsd(a, b):
    while b:
        a, b = b, a % b
    return a

def prostye_chyslo(chyslo, iteratsii=5):
    if chyslo <= 1 or chyslo % 2 == 0:
        return False
    if chyslo == 2:
        return True
    
    eksponenta, zalysok = 0, chyslo - 1
    while zalysok % 2 == 0:
        eksponenta += 1
        zalysok //= 2

    for _ in range(iteratsii):
        baza = random.randint(2, chyslo - 2)
        kandydat = pow(baza, zalysok, chyslo)
        if kandydat == 1 or kandydat == chyslo - 1:
            continue
        
        for _ in range(eksponenta - 1):
            kandydat = pow(kandydat, 2, chyslo)
            if kandydat == chyslo - 1:
                break
        else:
            return False
    return True

def modularna_oborotna(element, totyent):
    staryy_zalysok, zalysok = element, totyent
    staryy_koefitsiyent, koefitsiyent = 1, 0
    
    while zalysok:
        chastka = staryy_zalysok // zalysok
        staryy_zalysok, zalysok = zalysok, staryy_zalysok - chastka * zalysok
        staryy_koefitsiyent, koefitsiyent = koefitsiyent, staryy_koefitsiyent - chastka * koefitsiyent
    
    return staryy_koefitsiyent % totyent

### 2. За допомогою цієї функції згенерувати дві пари простих чисел p, q і p1, q1:

In [7]:
def zgheneruvaty_kluchovi_pary(dovzhyna_bita):
    chyslo_p = zgheneruvaty_proste_chyslo(dovzhyna_bita)
    chyslo_q = zgheneruvaty_proste_chyslo(dovzhyna_bita)
    
    while chyslo_p == chyslo_q:
        print(f"Chysla p i q odnakovi. Perhe nerelytuvaty q.")
        chyslo_q = zgheneruvaty_proste_chyslo(dovzhyna_bita)

    modul_n = chyslo_p * chyslo_q
    totyent_phi = (chyslo_p - 1) * (chyslo_q - 1)

    while True:
        publichnyy_eksponent_e = random.randint(3, totyent_phi - 1)
        if vyznachennya_nsd(publichnyy_eksponent_e, totyent_phi) == 1 and prostye_chyslo(publichnyy_eksponent_e):
            break

    private_eksponent_d = modularna_oborotna(publichnyy_eksponent_e, totyent_phi)

    return {'publichnyy_kluch': (publichnyy_eksponent_e, modul_n), 'private_kluch': (private_eksponent_d, chyslo_p, chyslo_q)}

### 3. Написати функцію генерації ключових пар для RSA.

In [8]:
def shifruvaty(plaintext, publichnyy_kluch):
    publichnyy_eksponent_e, modul_n = publichnyy_kluch
    return pow(plaintext, publichnyy_eksponent_e, modul_n)

def rozshifruvaty(ciphertext, private_kluch):
    private_eksponent_d, chyslo_p, chyslo_q = private_kluch
    modul_n = chyslo_p * chyslo_q
    return pow(ciphertext, private_eksponent_d, modul_n)

def pidpysaty(plaintext, private_kluch):
    private_eksponent_d, chyslo_p, chyslo_q = private_kluch
    modul_n = chyslo_p * chyslo_q
    return pow(plaintext, private_eksponent_d, modul_n)

def pereviryty(pidpys, publichnyy_kluch):
    publichnyy_eksponent_e, modul_n = publichnyy_kluch
    return pow(pidpys, publichnyy_eksponent_e, modul_n)


### 4. Написати програму шифрування, розшифрування і створення повідомлення з цифровим підписом для абонентів А і B.

In [9]:
def nadyslaty_kluch(spilnyy_kluch, publichnyy_kluch_otrymuvacha):
    return shifruvaty(spilnyy_kluch, publichnyy_kluch_otrymuvacha)

def otrymaty_kluch(enkryptyvanyy_spilnyy_kluch, private_kluch_otrymuvacha):
    return rozshifruvaty(enkryptyvanyy_spilnyy_kluch, private_kluch_otrymuvacha)


### 5. За допомогою раніше написаних на попередніх етапах програм організувати роботу протоколу конфіденційного розсилання ключів з підтвердженням справжності по відкритому каналу за допомогою алгоритму RSA.

In [10]:
def main():
    rozmir_kluchiv_bita = 256

    print("Zgheneruvannya kluchovykh par dlya Korysuvacha A ta Korysuvacha B...")
    kluchі_korysuvacha_A = zgheneruvaty_kluchovi_pary(rozmir_kluchiv_bita)
    kluchі_korysuvacha_B = zgheneruvaty_kluchovi_pary(rozmir_kluchiv_bita)
    
    print("Kluchi dlya Korysuvacha A:", kluchі_korysuvacha_A)
    print("Kluchi dlya Korysuvacha B:", kluchі_korysuvacha_B)

    random_message = random.randint(1, kluchі_korysuvacha_A['publichnyy_kluch'][1] - 1)
    print("\nRandomne povidomlennya:", random_message)

    enkryptyvanyy_dlya_A = shifruvaty(random_message, kluchі_korysuvacha_A['publichnyy_kluch'])
    rozshifruvanyy_dlya_A = rozshifruvaty(enkryptyvanyy_dlya_A, kluchі_korysuvacha_A['private_kluch'])
    print("Enkryptyvanyy dlya Korysuvacha A:", enkryptyvanyy_dlya_A)
    print("Rozshifruvanyy dlya Korysuvacha A:", rozshifruvanyy_dlya_A)

    enkryptyvanyy_dlya_B = shifruvaty(random_message, kluchі_korysuvacha_B['publichnyy_kluch'])
    rozshifruvanyy_dlya_B = rozshifruvaty(enkryptyvanyy_dlya_B, kluchі_korysuvacha_B['private_kluch'])
    print("Enkryptyvanyy dlya Korysuvacha B:", enkryptyvanyy_dlya_B)
    print("Rozshifruvanyy dlya Korysuvacha B:", rozshifruvanyy_dlya_B)

    pidpys_by_A = pidpysaty(random_message, kluchі_korysuvacha_A['private_kluch'])
    perevirka_by_A = pereviryty(pidpys_by_A, kluchі_korysuvacha_A['publichnyy_kluch'])
    print("\nTsifrovyy pidpys Korysuvacha A:", pidpys_by_A)
    print("Perevirka pidpysu dlya Korysuvacha A:", perevirka_by_A == random_message)
    
    print("\nPerevirka kluchiv vid Korysuvacha A do Korysuvacha B...")
    kluch_dlya_peredachi = random.randint(1, 1000)
    enkryptyvanyy_kluch_dlya_peredachi = nadyslaty_kluch(kluch_dlya_peredachi, kluchі_korysuvacha_B['publichnyy_kluch'])
    otrymanyy_kluch_dlya_peredachi = otrymaty_kluch(enkryptyvanyy_kluch_dlya_peredachi, kluchі_korysuvacha_B['private_kluch'])
    print("Peredavanyy kluch:", kluch_dlya_peredachi)
    print("Otrymanyy kluch:", otrymanyy_kluch_dlya_peredachi)


if __name__ == "__main__":
    main()


Zgheneruvannya kluchovykh par dlya Korysuvacha A ta Korysuvacha B...
Kandydat 97282216225087320878317026658811862385475783758082462970761930086530541273593 ne proste chyslo.
Kandydat 61579076099484412240309010445000605957937607007507397025632858520910283279791 ne proste chyslo.
Kandydat 72962114036583520281466657140798905080093210554877600967024779862805380735217 ne proste chyslo.
Kandydat 59948018344321513374192205828958675960234830459331635258732683716625860843897 ne proste chyslo.
Kandydat 107607413283537724595930865884829035863454042825431282114489694391220307298033 ne proste chyslo.
Kandydat 101869716813752681861251351104253588665464393152352079862925081347772787947443 ne proste chyslo.
Kandydat 65791794125834228367585253910706638270321559907921702432279367890749134112449 ne proste chyslo.
Kandydat 68266711250828623326689490202076391669006883506480368190305617211886916479423 ne proste chyslo.
Kandydat 108829856923909269430086980359172203787760678548882374842523954971807033816947 n